In [8]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
from datetime import datetime
import sys
import random
import time as t
import re
import os

path = "ekami"
path = "/Users/"+path+"/Downloads/filename"

In [9]:
def scrape(pages):

    job_list = []
    descr_list = []
    time_stamp = []
    payment_type = []
    duration = []
    workload = []
    experience = []
    budget = []

    for r in range(0,pages):
        #getting html soup
        url = "https://www.upwork.com/ab/jobs/search/?category2_uid=531770282580668418&page="+str(r)+"&sort=recency&user_location_match=1"
        agent = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
        source = rq.get(url, headers=agent).text
        soup = BeautifulSoup(source, 'html.parser')

        if "Access Denied" in soup.text:
            sys.exit("Access Denied \n Either User-Agent is incorrect or too many requests were made")
        else:
            print("Success")

        #finding each individual job post
        jobs = soup.findAll('section',{'class':"air-card air-card-hover job-tile-responsive"})

        for x in jobs:
            # print(x)
            job_list.append(x.find('up-c-line-clamp',{'lines':"2"}).text)
            descr_list.append(x.find('span',{"class":"js-description-text"}).text)


            time = x.time['datetime']
            time = datetime.fromisoformat(time)
            diff = datetime.utcnow()-time.replace(tzinfo=None)
            time_stamp.append(round(diff.total_seconds()/3600,2))

            #Project payment type, either hourly or fixed. If hourly rate separated
            payment = x.find('strong',{'class':'js-type text-muted'}).text
            if "Hourly" in payment:
                if "$" in payment:
                    ind = payment.rfind('$')
                    budget.append(float(payment[ind+1:].strip().strip('\n')))
                else:
                    budget.append(None)
                payment_type.append("Hourly")
            else:
                payment_type.append("Fixed")

            #Total number of months expected to complete project, only for hourly
            dur = x.find('strong',{'class':'js-duration'},text = True)
            if dur != None:
                duration.append(dur.text)
            else:
                duration.append(None)

            #Weekly hours need to commit, only for hourly projects
            wkload = x.find('strong',{'class':'js-workload'})
            if wkload != None:
                workload.append(wkload.text)
            else:
                workload.append(None)

            #Difficutly of project
            experience.append(x.find('strong',{'class':'js-contractor-tier'}).text)

            #Budget of fixed projects
            bug = x.find('strong',{'class':'js-budget'})
            if bug != None:
                bug = bug.find('span').text.strip()[1:].strip('\n')
                if ',' in str(bug):
                    budget.append(float(bug.replace(',','')))
                else:
                    budget.append(float(bug))
        t.sleep(15)
        print("Time: "+ str(.25*(r+1)) +" mins out of "+str(pages/4))

    df = pd.DataFrame({
        'Job Name':job_list,
        'Job Description':descr_list,
        'Payment Type':payment_type,
        'Budget':budget,
        'Workload':workload,
        'Level':experience,
        'Est. Time':duration,
        'Time Created':time_stamp
    })
    name = "Jobs.xlsx"
    try:
        os.remove(name)
        try:
            df = df.dropna(axis=0, subset=['Budget'])
            df.to_excel(name)
        except PermissionError:
            name = "Jobs"+str(random.randrange(100))+".xlsx"
            df = df.dropna(axis=0, subset=['Budget'])
            df.to_excel(name)

    except OSError:
        df = df.dropna(axis=0, subset=['Budget'])
        df.to_excel(name)
    # try:
    #     os.remove('Jobs.xlsx')
    #
    # except OSError:
    #     pass
    # df = df.dropna(axis=0, subset=['Budget'])
    # df.to_excel("Jobs.xlsx")
    return name

In [10]:
def filter(name):
    df = pd.read_excel(name)
    banned_words = ['drupal','wordpress','woocommerce']
    react = ['React Native','react','React']
    hardware = ['Hardware','embedded','microcontroller']
    scraping = ['Scrape','scraping']
    sports = ['sport','basketball','football','soccer','baseball']
    fixed_limit = 1000.00
    hourly_limit = 40.00

    drops = []
    c_react = []
    c_hardware = []
    c_scraping = []
    c_sports = []
    for index,row in df.iterrows():

        val = False
        for x in react:
            if (x.upper() in row['Job Description'].upper()) | (x.upper() in row['Job Name'].upper()):
                val = val | True
            else:
                val = val
        c_react.append(val)

        val = False
        for x in hardware:
            if (x.upper() in row['Job Description'].upper()) | (x.upper() in row['Job Name'].upper()):
                val = val | True
            else:
                val = val
        c_hardware.append(val)

        val = False
        for x in scraping:
            if (x.upper() in row['Job Description'].upper()) | (x.upper() in row['Job Name'].upper()):
                val = val | True
            else:
                val = val
        c_scraping.append(val)

        val = False
        for x in sports:
            if (x.upper() in row['Job Description'].upper()) | (x.upper() in row['Job Name'].upper()):
                val = val | True
            else:
                val = val
        c_sports.append(val)

        for x in banned_words:
            if (x.upper() in row['Job Description'].upper()) | (x.upper() in row['Job Name'].upper()):
                drops.append(index)

        if (row['Payment Type'] == "Fixed") & (float(row['Budget']) < fixed_limit):
            if index not in drops:
                drops.append(index)

        elif (row['Payment Type'] == "Hourly") & (float(row['Budget']) < hourly_limit):
            if index not in drops:
                drops.append(index)

    df['React Native'] = c_react
    df['Hardware'] = c_hardware
    df['Scrape'] = c_scraping
    df['Sports'] = c_sports
    df.drop(drops, inplace=True)
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(ignore_index=True)
    df = df.rename(columns={'Time Created':'Hours Since Posting'})
    df.sort_values(['React Native', 'Hardware','Scrape','Sports'], ascending=[False, False, False, False], inplace=True)
    name = path+"/Jobs_cleaned.xlsx"
    try:
        os.remove(name)
        df.to_excel(name,
                    columns=['Job Name', 'Job Description', 'Payment Type', 'Budget', 'Workload', 'Level',
                             'Est. Time', 'Hours Since Posting', 'React Native', 'Hardware', 'Scrape', 'Sports'])
    except PermissionError:
        name = "Jobs_cleaned"+str(random.randrange(100))+".xlsx"
        df.to_excel(name,
                        columns=['Job Name', 'Job Description', 'Payment Type', 'Budget', 'Workload', 'Level',
                                 'Est. Time', 'Hours Since Posting', 'React Native', 'Hardware', 'Scrape', 'Sports'])

    except OSError:
        df.to_excel("Jobs_cleaned.xlsx",columns=['Job Name', 'Job Description','Payment Type','Budget','Workload','Level',
                                             'Est. Time','Hours Since Posting','React Native','Hardware','Scrape','Sports'])
    return name

In [11]:
df = scrape(2)
filter(df)

Success
Time: 0.25 mins out of 0.5
Success
Time: 0.5 mins out of 0.5


'/Users/ekami/Downloads/filename/Jobs_cleaned.xlsx'